In [1]:
import cv2
import tensorflow
from tensorflow.keras.models import load_model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
trained_model = load_model("trained_models/model_tf_e.h5")

print("loaded model")
# Mapping of emotion indices to emotion labels
emotion_dict = {
    0: 'Surprise',
    1: 'Fear',
    2: 'Disgust',
    3: 'Happy',
    4: 'Sad',
    5: 'Angry',
    6: 'Neutral'
}
print("loaded emotion_dict")





loaded model
loaded emotion_dict


In [5]:
import cv2
import numpy as np
import time
import requests
import io
from PIL import Image

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print("loaded face cascade")
# Mapping of emotion indices to emotion labels
emotion_dict = {
    0: 'Surprise',
    1: 'Fear',
    2: 'Disgust',
    3: 'Happy',
    4: 'Sad',
    5: 'Angry',
    6: 'Neutral'
}
print("loaded emotion_dict")

def detect_emotion_from_api(frame):
    """Send frame to emotion detection API and get results"""
    try:
        # Convert frame to PIL Image
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(frame_rgb)
        
        # Convert to bytes
        img_byte_arr = io.BytesIO()
        pil_image.save(img_byte_arr, format='JPEG')
        img_byte_arr.seek(0)
        
        # Prepare the request
        files = {
            'file': ('image.jpg', img_byte_arr, 'image/jpeg')
        }
        
        # Make API call
        response = requests.post(
            'https://emotions-from-face-production.up.railway.app/emotion/detect',
            files=files,
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            if result.get('success') and 'emotion' in result:
                emotion = result['emotion']
                face_coords = result.get('face_coordinates', {})
                return emotion, face_coords
            else:
                return "Unknown", {}
        else:
            return "API Error", {}
            
    except requests.exceptions.Timeout:
        return "Timeout", {}
    except requests.exceptions.RequestException as e:
        return "Request Error", {}
    except Exception as e:
        return "Error", {}

# Function to classify emotions from detected faces using API
def classifyEmotionFromFace(face_cascade, frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        # Find the largest face detected
        biggest_face = max(faces, key=lambda rect: rect[2] * rect[3])
        x, y, width, height = biggest_face
        
        try:
            # Call API for emotion detection on the whole frame
            emotion, api_face_coords = detect_emotion_from_api(frame)
            return emotion, (x, y, width, height)
        except Exception as e:
            return "Error on classification", None
    else:
        return "No face detected", None

# Open camera and start emotion detection
cap = cv2.VideoCapture(0)
last_prediction_time = time.time()
current_emotion = "Starting..."

print("Camera opened. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Check if 2 seconds has passed since last prediction (to avoid too many API calls)
    current_time = time.time()
    if current_time - last_prediction_time >= 2.0:
        emotion, face_coords = classifyEmotionFromFace(face_cascade, frame)
        current_emotion = emotion
        last_prediction_time = current_time
        print(f"Detected emotion: {current_emotion}")
    
    # Draw face detection box and emotion text only for the biggest face
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        # Find and draw only the biggest face
        biggest_face = max(faces, key=lambda rect: rect[2] * rect[3])
        x, y, w, h = biggest_face
        
        # Draw rectangle around the biggest face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Add emotion text above the biggest face
        cv2.putText(frame, current_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Display the frame
    cv2.imshow('Emotion Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()
print("Camera released and windows closed.")


loaded face cascade
loaded emotion_dict
Camera opened. Press 'q' to quit.
Detected emotion: No face detected
Detected emotion: Surprise
Detected emotion: Surprise
Detected emotion: Happy
Detected emotion: Disgust
Detected emotion: Neutral
Detected emotion: Neutral
Detected emotion: Surprise
Detected emotion: Surprise
Detected emotion: Surprise
Detected emotion: Surprise
Detected emotion: Surprise
Detected emotion: Neutral
Detected emotion: Surprise
Detected emotion: Neutral
Detected emotion: Happy
Detected emotion: Happy
Detected emotion: Surprise
Detected emotion: Sad
Detected emotion: Sad
Detected emotion: Neutral
Detected emotion: Disgust
Detected emotion: Surprise
Detected emotion: Surprise
Detected emotion: Happy
Detected emotion: Happy
Detected emotion: Neutral
Detected emotion: Neutral
Detected emotion: Surprise
Detected emotion: Surprise
Detected emotion: Sad
Detected emotion: Sad
Detected emotion: Sad
Detected emotion: Surprise
Detected emotion: Neutral
Detected emotion: Surpr

In [2]:
# This error occurs when trying to load a TensorFlow/Keras model that was saved with an older version
# of TensorFlow/Keras and contains parameters that are no longer supported in the current version.

# The specific issue is with the DepthwiseConv2D layer which has a 'groups' parameter in the saved
# model configuration, but the current version of Keras doesn't recognize this parameter.

# This is a common compatibility issue when:
# 1. The model was trained/saved with TensorFlow 1.x or an older TensorFlow 2.x version
# 2. You're trying to load it with a newer version of TensorFlow/Keras
# 3. The layer parameters have changed between versions

# Solutions to try:
# 1. Use the same TensorFlow version that was used to save the model
# 2. Load with custom_objects to handle deprecated parameters
# 3. Convert the model to a newer format
# 4. Retrain the model with the current TensorFlow version